# Zero to Hero

## 1.07 Aplicando el arbol

Se aplicará el modelo generado por rpart a los datos nuevos y se genera la salida para Kaggle

In [1]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("C:/Users/IMR/Onedrive - profertil.com.ar/MCD/1 Año/Laboratorio de implementacion I/datasets")  #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,614665,32.9,1321211,70.6,1321211,70.6
Vcells,1093462,8.4,8388608,64.0,1632355,12.5


In [6]:
dataset <- fread("./dataset_pequeno.csv")   #cargo el dataset #cargo el dataset

In [7]:
dtrain <- dataset[ foto_mes==202107 ]
dapply <- dataset[ foto_mes==202109 ]

In [9]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                        data= dtrain,
                        xval= 0,
                        cp= -1,
                        maxdepth= 2 )

Aplico el modelo a los datos dapply  pidiendo que me devuelva probabildades

In [10]:
prediccion  <- predict( modelo, dapply, type = "prob")

Analizo que queda en prediccion

In [11]:
head( prediccion )

,BAJA+1,BAJA+2,CONTINUA
1,0.002285635,0.003102923,0.9946114
2,0.002285635,0.003102923,0.9946114
3,0.034907598,0.030458590,0.9346338
4,0.002285635,0.003102923,0.9946114
5,0.002285635,0.003102923,0.9946114
6,0.002285635,0.003102923,0.9946114


Prediccion es una matriz; me interesa la columna "BAJA+2"  que es la probabilidad que modelo asigna a cada registro de dapply

In [12]:
prob_baja2  <- prediccion[, "BAJA+2"]

In [13]:
head( prob_baja2)

1           2           3           4           5           6 
0.003102923 0.003102923 0.030458590 0.003102923 0.003102923 0.003102923

prob_baja2 es el vector de probabilidades

In [14]:
length( prob_baja2 )
nrow( dapply )

[1] 165237

[1] 165237

Ahora decido si envio el estimulo o no
<br>
si prob( BAJA+2) > 0.025  envio  el estímulo

In [15]:
Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )

In [16]:
head( Predicted)

1 2 3 4 5 6 
0 0 1 0 0 0

In [ ]:
hist( Predicted)

creo una tabla con  lo que voy a enviar a Kaggle

In [17]:
entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

In [18]:
head( entrega)

numero_de_cliente,Predicted
<int>,<dbl>
31116053,0
31116803,0
31117730,1
31117908,0
31117977,0
31120119,0


In [19]:
entrega[ , .N, Predicted]

Predicted,N
<dbl>,<int>
0,147829
1,17408


Esto significa que se enviaran 17408 estímulos

In [20]:
#genero el archivo para Kaggle
#creo la carpeta donde va el experimento
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/ZH2017/", showWarnings = FALSE )

fwrite( entrega, 
        file= "./exp/ZH2017/para_Kaggle_0107.csv",
        sep=  "," )